In [44]:
from src.DataPreparation import fc, ffc
import pandas as pd

In [2]:
fc.count_documents({})

1456

In [4]:
users_data = list(fc.find())

In [93]:
user_data_nested = users_data[1]

In [94]:
user_data_nested

{'_id': ObjectId('5bfc88f3baead13dc62256a5'),
 'url': 'https://www.instagram.com/lirlando/',
 'bundle_variant': 'base',
 'config': {'csrf_token': 'fDcjrRvCtFooD23xol5qBzr6moL3aY5i',
  'viewer': None,
  'viewerId': None},
 'country_code': 'US',
 'deployment_stage': 'c2',
 'entry_data': {'ProfilePage': [{'logging_page_id': 'profilePage_8882294111',
    'show_suggested_profiles': False,
    'graphql': {'user': {'biography': '',
      'blocked_by_viewer': False,
      'country_block': False,
      'external_url': None,
      'external_url_linkshimmed': None,
      'edge_followed_by': {'count': 63},
      'followed_by_viewer': False,
      'edge_follow': {'count': 76},
      'follows_viewer': False,
      'full_name': 'Irlando Lima',
      'has_channel': False,
      'has_blocked_viewer': False,
      'highlight_reel_count': 0,
      'has_requested_viewer': False,
      'id': '8882294111',
      'is_business_account': False,
      'is_joined_recently': False,
      'business_category_name':

In [100]:
user_data_nested['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['edges'][0]['node']['edge_liked_by']['count']

10

In [110]:
def flatten_user_data(user_data_nested):
    """Return a flat dictionary of user data."""
    udn = user_data_nested
    ud = {}
    ud['url'] = udn['url']
    if ('entry_data' in udn
        and 'ProfilePage' in udn['entry_data']
        and type(udn['entry_data']['ProfilePage']) == list
        and len(udn['entry_data']['ProfilePage']) > 0
        and 'graphql' in udn['entry_data']['ProfilePage'][0]
        and 'user' in udn['entry_data']['ProfilePage'][0]['graphql']
       ):
        gql = udn['entry_data']['ProfilePage'][0]['graphql']['user']
        ud['bio'] = gql['biography']
        ud['followed_by'] = gql['edge_followed_by']['count']
        ud['follows'] = gql['edge_follow']['count']
        ud['num_posts'] = gql['edge_owner_to_timeline_media']['count']
        ud['id'] = gql['id']
        ud['is_joined_recently'] = gql['is_joined_recently']
        ud['is_private'] = gql['is_private']
        ud['is_business_account'] = gql['is_business_account']
        
        if ('edge_owner_to_timeline_media' in gql
            and 'edges' in gql['edge_owner_to_timeline_media']
            and len(gql['edge_owner_to_timeline_media']['edges'])> 0
            and 'node' in gql['edge_owner_to_timeline_media']['edges'][0]
            and 'edge_liked_by' in gql['edge_owner_to_timeline_media']['edges'][0]['node']
            and gql['is_private'] == False
           ):
            ud['likes_last_post'] = gql['edge_owner_to_timeline_media']['edges'][0]['node']['edge_liked_by']['count']
        
        
    return ud
    

In [111]:
flatten_user_data(user_data_nested)

{'url': 'https://www.instagram.com/lirlando/',
 'bio': '',
 'followed_by': 63,
 'follows': 76,
 'num_posts': 2,
 'id': '8882294111',
 'is_joined_recently': False,
 'is_private': False,
 'is_business_account': False,
 'likes_last_post': 10}

In [112]:
def update_ffc():    
    for user in fc.find():
        if user['url'].endswith('/'):
            ffc.delete_many({'url': user['url']})
            user['url'] = user['url'][:-1]
        flat_user = flatten_user_data(user)
        ffc.delete_many({'url': flat_user['url']})
        ffc.insert_one(flat_user)

In [113]:
update_ffc()

In [114]:
df = pd.DataFrame(list(ffc.find()))
df.head()

,_id,bio,followed_by,follows,id,is_business_account,is_joined_recently,is_private,likes_last_post,num_posts,url
0,5c098e3fbaead1a820a3cfcc,,63.0,76.0,8882294111,False,False,False,10.0,2.0,https://www.instagram.com/lirlando
1,5c098e3fbaead1a820a3cfce,Haters make me famous😻\n@irvan740 ❤,870.0,136.0,7422647242,True,False,False,156.0,3.0,https://www.instagram.com/ndah.endah.3701
2,5c098e3fbaead1a820a3cfe4,Viaggia... a modo tuo! #Mazzone #Travel,2100.0,700.0,4091271053,True,False,False,26.0,180.0,https://www.instagram.com/mazzoneviaggi
3,5c098e3fbaead1a820a3cfe5,,14.0,48.0,9418121438,False,True,False,NaN,0.0,https://www.instagram.com/chrismainahperre
4,5c098e3fbaead1a820a3cff1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/lass.chuletitasss.papa


In [115]:
df.shape

(1270, 11)

In [47]:
from collections import Counter
bio_counts = Counter(df['bio'])


In [48]:
bio_counts.most_common(n=5)

[('', 521),
 (nan, 75),
 ('Haters make me famous😻\n@irvan740 ❤', 1),
 ('Viaggia... a modo tuo! #Mazzone #Travel', 1),
 ('@kha10lid_n', 1)]

In [49]:
df[df['bio'] == ""].head(10)

,_id,bio,external_url,followed_by,follows,id,is_business_account,is_joined_recently,is_private,num_posts,url
0,5c0981aabaead1a820a3be62,,None,63.0,76.0,8882294111,False,False,False,2.0,https://www.instagram.com/lirlando
3,5c0981aabaead1a820a3be7b,,None,14.0,48.0,9418121438,False,True,False,0.0,https://www.instagram.com/chrismainahperre
10,5c0981acbaead1a820a3bee1,,None,0.0,8.0,4854633336,False,False,True,0.0,https://www.instagram.com/lem0ns33
13,5c0981acbaead1a820a3bf03,,None,42.0,52.0,9418832835,False,True,False,1.0,https://www.instagram.com/ng_fashionteam
14,5c0981acbaead1a820a3bf11,,None,4726.0,227.0,5385360031,True,False,False,9.0,https://www.instagram.com/shyrose_cut
16,5c0981adbaead1a820a3bf2b,,http://www.fujima.com.mx/,3.0,27.0,9371188278,True,True,False,0.0,https://www.instagram.com/fujima_sacv
18,5c0981adbaead1a820a3bf2d,,None,31.0,313.0,9383339496,False,True,False,1.0,https://www.instagram.com/youssef.milan.1293
19,5c0981adbaead1a820a3bf2e,,None,56.0,430.0,9264036297,False,True,False,1.0,https://www.instagram.com/emuaehi7
20,5c0981adbaead1a820a3bf2f,,None,5.0,566.0,9290995463,False,True,False,1.0,https://www.instagram.com/demirturk_sarp
21,5c0981adbaead1a820a3bf30,,None,14.0,8.0,9307361373,False,True,False,0.0,https://www.instagram.com/je.an7602


In [50]:
df.shape

(1270, 11)